In [1]:
! pip install datasets
! pip install rouge_score

In [2]:
from datasets import load_dataset

vals_ds = load_dataset('json', data_files='/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', split='train[0:2500]')
trains_ds = load_dataset('json', data_files='/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', split='train[2500:27500]')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5b9a3c3b7f964b2a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


In [3]:
from transformers import pipeline, AutoTokenizer, TFAutoModelForSeq2SeqLM

model_name ='t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
# summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

2022-04-24 05:27:27.382640: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 05:27:27.383919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 05:27:27.384690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 05:27:27.385646: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
prefix = "summarize: "


def preprocess_function(examples):
    
    inputs = [prefix + doc for doc in examples["abstract"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [5]:
train_tokenized = trains_ds.map(preprocess_function, batched=True)
test_tokenized = vals_ds.map(preprocess_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [6]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [7]:
tf_train_set = train_tokenized.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = test_tokenized.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [8]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [9]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [10]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

2022-04-24 05:27:58.031749: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3
1562/1562 [==============================] - 521s 321ms/step - loss: 2.6775 - val_loss: 2.3080
Epoch 2/3
1562/1562 [==============================] - 496s 317ms/step - loss: 2.4905 - val_loss: 2.2383
Epoch 3/3
1562/1562 [==============================] - 498s 319ms/step - loss: 2.4174 - val_loss: 2.1956


In [ ]:
model.save_pretrained("./summarizer_model")

In [ ]:
from IPython.display import FileLink
FileLink(r'./summarizer_model/tf_model.h5')

In [ ]:
from datasets import load_metric

metric = load_metric("rouge")
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
# from datasets import load_metric

# metric = load_metric("rouge")
candidates = 'He had gone to the market'
references = 'He had gone to the market'
result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
# result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
print(result)

In [11]:
test_ds = load_dataset('json', data_files='/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', split='train[30000:30100]')


In [22]:
candidate_summaries = []

for text in test_ds:
    input_ids = tokenizer(text['abstract'], truncation=True, padding='longest', return_tensors="tf").input_ids
    output = model.generate(input_ids, max_length=20)
    predictions = tokenizer.batch_decode(output, skip_special_tokens=True)
    candidate_summaries.append(predictions[0])


0


In [24]:
from datasets import load_metric

metric = load_metric("rouge")
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [26]:
calc_rouge_scores(candidate_summaries, test_ds['title'])

{'rouge1': 34.8, 'rouge2': 18.2, 'rougeL': 31.2, 'rougeLsum': 31.6}